## Phylogenetic models in R

### 1. Introduction and resources

This practical should be a refresher on linear models in `R`, before introducing you to a phylogentic least squares model, or a PGLS. Because species that are closely related often share similar traits, this means we can't treat them as statistically independent. However, if we look at how the traits are spread throughout the tree, we can 'control' for this non-independance. 
We'll go into more detail when we run our PGLS. 

### 2. Linear models

For this pratical we'll be working data from the family Anatidae (ducks) to investigate Bergmann’s rule, if there is a relationship between latitude and body mass. 
First, we'll load in the data and inspect it:

In [ ]:
dir.create("My Git Repo")
git2r::clone("https://github.com/Syrph/BCB_Practicals", "My Git Repo")
setwd("My Git Repo")
system("sudo apt-get install libgdal-dev libproj-dev libgeos-dev libudunits2-dev libv8-dev libprotobuf-dev libjq-dev")
source("install.R")

In [ ]:
# Load the duck latitudinal and bodymass data
duck_data <- read.csv("duck_data.csv",h=T) 

# Check it's been imported
str(duck_data)
head(duck_data)

# Remove any NAs in the data (make sure to check you're not loosing too much data!)
duck_data <- na.omit(duck_data)

The midpoint latitude is the center of the distribution of each species. Because we're interested in the distance from equator, we'll use the `abs()` function to convert our data.

In [ ]:
duck_data$abs_latitude <- abs(duck_data$Latitude)

We'll start by looking at the relationship between body mass and latitude using a scatterplot.

In [ ]:
plot(Body_Mass ~ abs_latitude, data = duck_data)

Now there doesn't seem to be much of a relationship at all from our plot. However, to double check we should look at the spread of data for both variables. In particular, body mass is often scaled logarithmically, with lots of small species and fewer large ones. Therefore we might not be seeing the true relationship! 

In [ ]:
# We'll use a histogram to look at the spread.
hist(duck_data$Body_Mass)

As we suspected! The histogram suggests a log-normal distribution. If we take logs we might see a more normal distribution.

In [ ]:
duck_data$log_BM <- log(duck_data$Body_Mass)
hist(duck_data$log_BM)

Now we've got some data that resembles a more normal distribution! We'll now look at the spread of latitude.

In [ ]:
hist(duck_data$abs_latitude)

Not great, but no obvious signs of left or right skews in the data, so we can work with it. We'll leave it as it is.
Let's look at the new relationship between the two variables:

In [ ]:
plot(log_BM ~ abs_latitude, data = duck_data)

Now we're starting to see some kind of relationship! There's a lot of spread to the points, but we can see the smallest species at the lowest latitudes, and the largest at the highest. To really find out if there's a relationship we can test our hypothesis with a linear model. 

In [ ]:
# Run a basic linear model. We separate our dependant variables from predictors using a tilda ~
duck_model <- lm(log_BM ~ abs_latitude, data = duck_data)

# Inspect our linear model
summary(duck_model)

Now we can investigate if there is a relationship. There's quite a lot going on with our output, but for this practical we'll focus on just a few main things:

`Coefficients`: This tells us about our predictors in the model. In this one there's 2, the intercept, and latitude.  We'll break each section down further.

`Estimate`: The estimate of our coefficients tells us what value it should have. For the intercept this will be the point that that crosses across the y axis. For latitude, this will be the gradient of the relationship between latitude and body mass. 

`Std. Error`: This shows how much faith we have in our estimates. We're fairly certain that our estimates will fall within the range: Estimate +- Standard Error. 

`t value`: This is our test statistic. In a linear model we're testing if each of our estimate values are significantly different from zero. If our estimate and standard error don't overlap zero, it normally means they are significant. 

`Pr(>|t|)`: This is our p values for each predictor. This is calculated by weighing up the degrees of freedom against our test statistic, and tells us what the chance is that we observed the same pattern in our data given that there was no relationship, i.e. the null hypothesis is true. 

`Multiple R-squared`: This tells us how much of the variation in our response variable is explained by our model. Large values are better, but often in macro-evolution we see smaller values. Because traits at a macro scale are often driven by multiple selection pressures, which may sometimes be species-specific, we typically explain less variation than smaller more targeted studies. 

`Adjusted R squared`: This also explains the varition in response, but penalises us for including more predictors. This reduces the chances of over-fitting models with lots of predictors that don't contribute much. This is the R-squared that tends to be reported in publications. 

`F Statistc` & `DF` & `p-value`: The last line reports the overall results of our model. When reporting the statistic tests in the results section, we tend to quote these values for the model. This test is comparing our model line against a flat horizontal line at the mean body mass. Simply put, does our latitude model explain more of the variance in body mass than the mean. This is easiest to explain with a quick example:

In [ ]:
# Create some data
x <- c(12,18,21, 36, 44, 54, 59)
y <- c(2, 4, 7, 11, 12, 14, 15)

# Create a linear model based only on the mean of body mass
mean <- lm(y ~ 1)

# Create a linear model where x predicts y
linear <- lm(y ~ x)

# Create a plot window with one row and two columns
par(mfrow =c (1,2))

# Plot our data for the mean
plot(x,y, xlim = c(0,60), ylim =c(0,15), main = "Mean") 

# Add the line of the linear model based on the mean
abline(mean, col="red")

# Add in lines to show the distance from each point to mean line (the residuals)
segments(x, y, x, predict(mean))

# Do the same to plot our data with the linear model based on x
plot(x,y, xlim = c(0,60), ylim =c(0,15), main = "Linear")  
abline(linear, col="blue")
segments(x, y, x, predict(linear))

From the plots we can see that the blue linear model line passes closer to all of our data points than simply using the mean line. The black lines from our data points to the linear model are the residual variation left over once we've accounted for x. This is often referred to as the residuals.

The F statistic in our summary output is testing if there is a siginificant difference between the residuals from using our mean line against using our linear model instead. This is weighed up against the number of degrees of freedom to calculate our p-value. 

Degrees of freedom are often poorly known but are actually quite simple to understand. They are calculated from the number of independent data points in your model, minus the number of predictors. This is to prevent models that over-fit the data. So models with lots of data points have high degrees of freedom which means we need lower F statistic values to be certain of our model. For models with few data points it depends on the number of predictors. If there's few predictors, like in our model, that means that we can accept lower F statistics. We can be more confident in our relationship if we used fewer predictors to describe it. If we use lots of predictors, we can be less certain in our model, because each predictor may explain some of the variation just by chance. Therefore we need a higher F statistic. When you report your models, report both the degrees of freedom and the F statistic alongside your p-value for the whole model. 

Now that we understand a bit more about our summary report, lets look at it again to investigate the relationship between body mass and latitude. 

In [ ]:
summary(duck_model)

We can see from our model that both the intercept and latitude are significant predictors. That the intercept is significant isn't very interesting. It means at 0 latitude (the equator), body mass is significantly different from zero. Seeing as it's impossible to have a species with zero body mass, this isn't suprising! What's more interesting is latitude. We can see a significant p-value, so there is a relationship between latitude and body mass. As the estimate is postive, we can see that as latitude increases, so does body mass. For every 1 degree of latitude, log(body mass) increases by 0.012. We can simply convert our body mass back into normal units for a more intuitive value: 

In [ ]:
# Get the exponential of our estimate
exp(0.011639)

So for every one degree latitude increase, our model predicts a 1g increase in body weight, supporting Bergmann’s rule. We can see that by plotting our model line with our data.

In [ ]:
plot(log_BM ~ abs_latitude, data = duck_data)
abline(duck_model)

Of course, we can see that the spread of data that many points don't fit this line. If we look at the adjusted R-squared, we can see that our model explains roughly 9% of the variation in body size. Most macroevolutionary studies have low R-squared values, so this is quite high! We could potentially increase this more by including other predictors which influence body size. Have a think about what these predictors could be. 

We can also see from the bottom line of output that our overall model is significant. Because there is only one predictor (except the intercept), this value will be the same as our p-value for body mass.

As we've ran a standard linear model, we should also check our residuals to see if they are normally distributed. This is one of the assumptions of parametric tests, and if not we might consider using a generalised linear model instead. 

In [ ]:
# Plot a density curve of the residuals
plot(density(duck_model$residuals))

Our residuals look pretty normally distributed. It's normally good enough just to inspect these plots by eye, to check there's no extreme left or right skew to the distribution. 

### 3. Phylogenetic Generalised Least Squares Models

Up until now we have been treating all our species as independant data points. However, technically this isn't true. Each species is related to each other, and some are more closely related than others. We might expect that closely related species in the same genus are more likely to have a similar body mass than species from different genera. We then come to the conclusion that there are more large species at higher latitudes because they all shared one common ancestor, who happened to be a large species. This would suggest that the evolutionary history of ducks is responsible for the patterns of body mass, rather than a true relationship between latitude and body mass. Fortunately, we can test this using phylogenetically-controlled linear models. One of the easiest to use a PGLS. 

First let's load up the packages we need and the phylogenetic tree of ducks.

In [ ]:
library(ape)
library(caper)

# Read in the tree
duck_tree <- read.tree("duck_tree.tre")
plot(duck_tree)

We now need to attach our body mass data and tree together, and we can do this by creating a comparative data object from the caper package.

In [ ]:
# We need to change the Jetz names so that they match the tip labels. gsub is another function to swap patterns in strings.
duck_data$Jetz_Name <- gsub(" ", "_", duck_data$Jetz_Name)

# We specify the phylogeny we need, the data, and which column has the tip label names in
duck_comp <- comparative.data(phy = duck_tree, data = duck_data, names.col = "Jetz_Name")

We can inspect our comparative data object to check that it's worked. 

In [ ]:
# Return the data 
duck_comp$data

In [ ]:
# Plot the phylogeny
plot(duck_comp$phy)

So we can see that our comparative object has worked as it should. Now we can run a pgls to see if information on the phylogeny makes any difference.

In [ ]:
duck_pgls <- pgls(log_BM ~ abs_latitude, data = duck_comp, lambda = "ML")

The code for a pgls looks largely the same. The only difference is that we have a third arguement, which is the lambda value. The lambda value tells us how randomly body mass and latitude are spread throughout the tree. By saying `"ML"` we've asked the function to calculate lambda using maximum likelihood methods, rather than give it an exact value. 

Let's take a look at the results of the pgls.

In [ ]:
summary(duck_pgls)

The output of the summary look largely the same as our linear model. The key difference is we now have information on the branch length transformations, which shows how our trait is influenced by phylogeny. We can also see that our p-value for latitude is now much higher, and above the 0.05 threshold. When we look at our estimate, we can see that it's a postive value, so the same relationship is there, but we can no longer be confident enough to reject our null hypothesis. This is why for macro-evolutionary studies, we always have to include information on the phylogeny! 

We should take a second to look at the lambda value. We can plot the profile of our lambda model and see how we came to this number.

In [ ]:
# Get the potential values of lambda
lambda_likelihood <- pgls.profile(duck_pgls, which = "lambda")

# Plot them
plot(lambda_likelihood)

On the horizontal axis we can see potential lambda values, and on the vertical is how likely they are. Red lines show the 95% confidence intervals. This shows that we are fairly confident in our lambda value. It's always worth plotting the our lambda profile, as a flatter line would mean we're less confident in our lambda, and might not have controlled for our phylogeny properly. But what does the lambda value actually do? 

Lambda is scaled between 0 and 1, and it's easiest to think of it as how much our trait is bunched up in the tree. Values closer to zero suggest that body mass would be spread randomly among the tree, and the phylogeny does not matter. Values closer to one suggest that body mass is organised strongly throughout the tree, with closer species having more similar sizes. 

For an excellent explanation of lambda values, check out this paper by Natalie Cooper at the Natural History Museum, who helped write the second practical on this course. 

https://royalsocietypublishing.org/doi/full/10.1098/rstb.2012.0341 

What the lambda value actually does is change the length of the branches on the tree, to reflect how body mass is related between species. We can visualise this by plotting trees with different lambda values. 

In [ ]:
# Load the package geiger that has the rescale function. You'll have to install it if you're in Rstudio on your own laptops.
library(geiger)

# We'll create six trees with different lambda values 
lambda_1_tree <- rescale(duck_tree, "lambda", 1)
lambda_0.8_tree <- rescale(duck_tree, "lambda", 0.8)
lambda_0.6_tree <- rescale(duck_tree, "lambda", 0.6)
lambda_0.4_tree <- rescale(duck_tree, "lambda", 0.4)
lambda_0.2_tree <- rescale(duck_tree, "lambda", 0.2)
lambda_0_tree <- rescale(duck_tree, "lambda", 0)

# Now we'll plot them alongside each other to see the difference

# Change the number of plots and resize the window
par(mfrow = c(2,3))
options(repr.plot.width=15, repr.plot.height=15)

plot(lambda_1_tree, show.tip.label = FALSE, direction = "downwards", main = "1.0")
plot(lambda_0.8_tree, show.tip.label = FALSE, direction = "downwards", main = "0.8")
plot(lambda_0.6_tree, show.tip.label = FALSE, direction = "downwards", main = "0.6")
plot(lambda_0.4_tree, show.tip.label = FALSE, direction = "downwards", main = "0.4")
plot(lambda_0.2_tree, show.tip.label = FALSE, direction = "downwards", main = "0.2")
plot(lambda_0_tree, show.tip.label = FALSE, direction = "downwards", main = "0.0")

What's actually happening is the lambda value shortens all the interal branches (everything except the tips). This reduces the difference between species. In the last plot we can see a lambda value of zero, and all the branches are equally close to the root, and therefore each other. This means that all our species are now independent points, and if we ran a pgls we would get similar results to a linear model. Try it out running a pgls with different lambda values and see what what happens!

Don't worry if you struggled to understand any of this! Lambda values can be tricky to get your head around. At this stage, it's only important to be aware that a pgls uses a lambda value to decide how much to weight up the importance of the phylogeny. 